In [1]:
import os
import torch as T
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.distributions.normal import Normal
import numpy as np
import matplotlib.pyplot as plt
import random



from gym.envs.classic_control import rendering



In [2]:
class ReplayBuffer(object):
    def __init__(self, mem_size, input_shape, n_actions):
        self.mem_size = mem_size
        self.n_actions = n_actions
        self.mem_cntr = 0
        self.state_memory = np.zeros((self.mem_size,*input_shape))
        self.new_state_memory = np.zeros((self.mem_size,*input_shape))
        self.action_memory = np.zeros((self.mem_size,n_actions))
        self.reward_memory = np.zeros(self.mem_size)
        self.terminal_memory = np.zeros(self.mem_size,dtype=np.bool_)
    def store_transition(self, state, action, reward,new_state, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index]= state
        self.new_state_memory[index]= new_state
        self.action_memory[index]= action #action is an array, so array of arrays
        self.reward_memory[index]= reward
        self.terminal_memory[index]= done
        self.mem_cntr += 1
    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr,self.mem_size) # cntr can be greater than size, but don't want to take mems that don't exist
        batch = np.random.choice(max_mem,batch_size) # from 0 to max_mem of size batch_size
        
        states = self.state_memory[batch]
        new_states = self.new_state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        terminals = self.terminal_memory[batch]
        
        return states,actions,rewards,new_states,terminals
    

In [3]:
class CriticNetwork(nn.Module):
    '''this evaluates the value of a state,action pair'''
    def __init__(self,beta, input_dims, n_actions,fc1_dims=256, fc2_dims=256,
                name='Critic Network',chkpt_dir='/Users/bookerschelhaas/Desktop/Code For fun/RL/SAC'):
        super(CriticNetwork,self).__init__()
        self.beta = beta #learning rate
        self.input_dims= input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.name = name
        self.chkpt_dir = chkpt_dir
        self.chkpt_file = os.path.join(self.chkpt_dir, name +'_sac')
        
        self.fc1 = nn.Linear(self.input_dims[0]+n_actions,self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims,self.fc2_dims)
        self.q = nn.Linear(self.fc2_dims, 1)
        
        self.optimizer = optim.Adam(self.parameters(), lr=self.beta)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
    def forward(self,state, action):
        action_value = self.fc1(T.cat([state,action],dim=1))
        action_value = F.relu(action_value)
        action_value = self.fc2(action_value)
        action_value = F.relu(action_value)
        q = self.q(action_value)
        return q
    def save_checkpoint(self):
        print('.....saving checkpoint.....')
        T.save(self.state_dict(), self.chkpt_file)
    def load_checkpoint(self):
        print('.......loading checkpoint........')
        self.load_state_dict(T.load(self.chkpt_file))
    

In [4]:
class ValueNetwork(nn.Module):
    '''Just estimates the value of a state or set of states,
    doesnt care about what action you took or are taking'''
    def __init__(self, beta, input_dims, fc1_dims=256,fc2_dims = 256,
                name='Value Network',chkpt_dir='/Users/bookerschelhaas/Desktop/Code For fun/RL/SAC'):
        super(ValueNetwork,self).__init__()
        self.beta = beta
        self.input_dims= input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.name = name
        self.chkpt_dir = chkpt_dir
        self.chkpt_file = os.path.join(self.chkpt_dir, name +'_sac')
        
        self.fc1 = nn.Linear(*self.input_dims,self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims,self.fc2_dims)
        self.v = nn.Linear(self.fc2_dims, 1)
        
        self.optimizer = optim.Adam(self.parameters(), lr=self.beta)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
    def forward(self,state):
        state_value = self.fc1(state)
        state_value = F.relu(state_value)
        state_value = self.fc2(state_value)
        state_value = F.relu(state_value)
        v = self.v(state_value)
        return v
    
    def save_checkpoint(self):
        print('.....saving checkpoint.....')
        T.save(self.state_dict(), self.chkpt_file)
    def load_checkpoint(self):
        print('.......loading checkpoint........')
        self.load_state_dict(T.load(self.chkpt_file))
    
        

In [5]:
class ActorNetwork(nn.Module):
    '''Outputs a mean and stdev distribution of actions'''
    def __init__(self,alpha, input_dims,max_action,fc1_dims=256,fc2_dims = 256,
                n_actions = 2,name='actor',chkpt_dir='/Users/bookerschelhaas/Desktop/Code For fun/RL/SAC'):
        '''max_action is the mulptiplication scalar so that the 
        actions in the environment are of the right scale'''
        super(ActorNetwork,self).__init__()
        self.alpha = alpha
        self.input_dims= input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.chkpt_dir = chkpt_dir
        self.n_actions =n_actions
        self.max_action = max_action
        self.reparam_noise = 1e-6 # can't take log of 0, 
        self.chkpt_file = os.path.join(self.chkpt_dir, name +'_sac')
        self.normal_tracker = []
        self.fc1 = nn.Linear(*self.input_dims,self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims,self.fc2_dims)
        self.mu = nn.Linear(self.fc2_dims,self.n_actions) #output is mean of probability distribution of each action for the policy
        self.sigma = nn.Linear(self.fc2_dims, self.n_actions)#output is stdev of probability distribution of each action for the policy
        
        self.optimizer = optim.Adam(self.parameters(), lr=self.alpha)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
    def forward(self,state):
        prob = self.fc1(state)
        prob = F.relu(prob)
        prob = self.fc2(prob)
        prob = F.relu(prob)
        
        mu = self.mu(prob)
        sigma = self.sigma(prob)
        # the clamp makes it so that the std dev isn't so wide.
        sigma = T.clamp(sigma,min = self.reparam_noise, max=1)
        
        return mu, sigma
    def sample_normal(self, state, reparameterize = True):
        mu, sigma = self.forward(state)
        self.normal_tracker.append((mu,sigma,state))
        #print('MU:',mu, 'sigma ', sigma)
        probabilites = Normal(mu, sigma)
        
        if reparameterize:
            actions = probabilites.rsample() #this gives a sample + some noise to encourage exploration
        else:
            actions = probabilites.sample()
        # tanh puts the value between -1 and 1, then multiply that by max value of action space to scale
        action_tanh = T.tanh(actions)
        action_scaled = T.tanh(actions) * T.tensor(self.max_action).to(self.device)
        log_probs = probabilites.log_prob(actions)  # for the loss function for updating network
        # .pow() is same as ** but tensor
        log_probs -= T.log(1-action_tanh.pow(2)+self.reparam_noise) # need the noise because log(0) = UNDEF
        log_probs = log_probs.sum(1,keepdim=True) # need a scalar quality to calculate loss 
        
        return action_scaled, log_probs
    
    
    def save_checkpoint(self):
        print('.....saving checkpoint.....')
        T.save(self.state_dict(), self.chkpt_file)
    def load_checkpoint(self):
        print('.......loading checkpoint........')
        self.load_state_dict(T.load(self.chkpt_file))
        

In [6]:
class SACAgent():
    '''Reward scale is for rewards and critic loss. takes into account 
    the entropy in system. Can be messed around with.
    
    tau is for doing the soft copy of the networks, instead of a 
    hard copy like in DQL
    '''
    def __init__(self,alpha = 0.0003, beta = 0.0003, input_dims = [1],
                 env = None, gamma = 0.99,n_actions=2, max_size = 1000000,tau = 0.005,
                layer1_size = 256, layer2_size = 256, batch_size = 256,
                reward_scale = 1):
        self.alpha = alpha
        self.beta = beta
        self.tau = tau
        self.input_dims = input_dims
        self.env = env
        self.gamma = gamma
        self.n_actions = n_actions
        self.max_size = max_size
        self.layer1_size = layer1_size
        self.layer2_size = layer2_size
        self.batch_size = batch_size
        self.scale= reward_scale
        
        self.memory = ReplayBuffer(max_size, input_dims, n_actions = n_actions)
        
        self.actor = ActorNetwork(self.alpha, self.input_dims,n_actions = n_actions,
                                  name = 'actor', max_action = env.action_space.high.item())
        self.critic_1 = CriticNetwork(beta,input_dims,
                                      n_actions = n_actions, name = 'critic 1')
        
        self.critic_2 = CriticNetwork(beta,input_dims,
                                      n_actions = n_actions, name = 'critic 2')
        self.value = ValueNetwork(beta, input_dims,name = 'value')
        self.target_value = ValueNetwork(beta,input_dims, 
                                         name = 'target value')
        
        self.update_network_parameters(tau=1) #does a hard copy of value network to target network on first time. Otherwise we will detune by tau
        
    def choose_action(self,state):
        ''' returns a numpy array of the mean of distribution for each action'''
        state = T.tensor([state],dtype=T.float).to(self.actor.device)
        actions, _ = self.actor.sample_normal(state, reparameterize = False)
        
        return actions.cpu().detach().numpy()[0]
    def remember(self,state, action , reward, new_state, done):
        self.memory.store_transition(state, action, reward, new_state, done)
        
    def update_network_parameters(self, tau = None):
        '''updates the target_value network to be soft copy of value netork'''
        if tau is None: #this makes it so it does a soft copy for n=2 onward
            tau = self.tau
        
        target_value_params = self.target_value.named_parameters() #gets current params for target value network
        value_params = self.value.named_parameters() # gets current vals for params for value network
        
        target_value_state_dict = dict(target_value_params)
        value_state_dict = dict(value_params)
        
        for name in value_state_dict:
            value_state_dict[name] = tau*value_state_dict[name].clone() +\
            (1-tau)*target_value_state_dict[name].clone()
        self.target_value.load_state_dict(value_state_dict)
    
    def save_models(self):
        print('.....saving models......')
        self.actor.save_checkpoint()
        self.critic_1.save_checkpoint()
        self.critic_2.save_checkpoint()
        self.value.save_checkpoint()
        self.target_value.save_checkpoint()
            
    def load_models(self):
        print('.....saving models......')
        self.actor.load_checkpoint()
        self.critic_1.load_checkpoint()
        self.critic_2.load_checkpoint()
        self.value.load_checkpoint()
        self.target_value.load_checkpoint()
    
    def learn(self):
        if self.memory.mem_cntr < self.batch_size:
            return
        
        state, action, reward, new_state, done = \
                    self.memory.sample_buffer(self.batch_size)
        
        rewards = T.tensor(reward,dtype=T.float).to(self.actor.device)
        dones = T.tensor(done).to(self.actor.device)
        new_states = T.tensor(new_state,dtype=T.float).to(self.actor.device)
        action = T.tensor(action,dtype=T.float).to(self.actor.device)
        states = T.tensor(state,dtype=T.float).to(self.actor.device)
        
        value = self.value(states).view(-1) #makes the datatype correct
        target_value = self.target_value(new_states).view(-1) #print this out without view and see what it does
        target_value[done] = 0.0 #???
        
        actions, log_probs = self.actor.sample_normal(states, reparameterize = False)
        log_probs = log_probs.view(-1)
        
        q1_new_policy = self.critic_1.forward(states,actions)
        q2_new_policy = self.critic_2.forward(states,actions)
        # take the min of the two q values. Stabilizes learning
        critic_value = T.min(q1_new_policy,q2_new_policy) 
        critic_value= critic_value.view(-1)
        
        self.value.optimizer.zero_grad()
        value_target = critic_value - 0.2*log_probs
        value_loss = 0.5* F.mse_loss(value,value_target)
        value_loss.backward(retain_graph =True)
        self.value.optimizer.step()
        
        actions, log_probs = self.actor.sample_normal(states, reparameterize = True) #reparameterize because we dont waant to lose gradient to update actor
        log_probs = log_probs.view(-1)
        q1_new_policy = self.critic_1.forward(states,actions)
        q2_new_policy = self.critic_2.forward(states,actions)
        #print(q1_new_policy,q2_new_policy, actions)
        # take the min of the two q values. Stabilizes learning
        critic_value = T.min(q1_new_policy,q2_new_policy) 
        critic_value= critic_value.view(-1)
        
        actor_loss = 0.2*log_probs - critic_value
        actor_loss = T.mean(actor_loss)
        self.actor.optimizer.zero_grad()
        actor_loss.backward(retain_graph = True)
        self.actor.optimizer.step()
        
        self.critic_1.optimizer.zero_grad()
        self.critic_2.optimizer.zero_grad()
        # q_hat includes entropy in loss function
        q_hat = self.scale*rewards + self.gamma*target_value
        q1_old_policy = self.critic_1.forward(states,action).view(-1)
        q2_old_policy = self.critic_2.forward(states,action).view(-1)
        critic_1_loss = 0.5 * F.mse_loss(q1_old_policy, q_hat)
        critic_2_loss = 0.5 * F.mse_loss(q2_old_policy, q_hat)
        
        critic_loss = critic_1_loss + critic_2_loss
        critic_loss.backward()
        self.critic_1.optimizer.step()
        self.critic_2.optimizer.step()
        
        self.update_network_parameters()
        
        
        
        

In [7]:
import pybullet_envs
import pybullet as p

p.connect(p.DIRECT)
import gym

#env = InvertedPendulum()
env = gym.make('MountainCarContinuous-v0')
#env = gym.make("BipedalWalker-v3")
#env = gym.make('InvertedDoublePendulumBulletEnv-v0')

agent = SACAgent(input_dims = env.observation_space.shape, env =env,
                n_actions = env.action_space.shape[0])

seed = np.random.seed(0) #have to do this

file_name = 'kfjdlkjf'
figure_file = 'plots/' +file_name
load_checkpoint = True

best_score = env.reward_range[0]
score_history = []
n_games = 50000
avg_scores = []
if load_checkpoint:
    agent.load_models()
    #env.render(mode='human')

for i in range(n_games):
    done = False
    score =0
    obs = env.reset()
    obs = env.observation_space.sample()
    print(obs)
    steps = 0
    while not done:
        act = agent.choose_action(obs)
       
        new_state, reward, done, info = env.step(act)
        agent.remember(obs,act,reward,new_state,done)
        if not load_checkpoint:
            agent.learn()
        score += reward
        obs = new_state
        steps+=1

    score_history.append(score)
    avg_score = np.mean(score_history[-100:])
    avg_scores.append(avg_score)
    
    if i %10 ==0:
        agent.save_models()
    if avg_score==best_score:
        best_score = avg_score
        if not load_checkpoint:
            agent.save_models()
    print('episode ', i,'score: ', score,'avg score: ', avg_score)

.....saving models......
.......loading checkpoint........
.......loading checkpoint........
.......loading checkpoint........
.......loading checkpoint........
.......loading checkpoint........
[-0.98968554 -0.02885164]


<ipython-input-6-970f0fc62b34>:42: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  state = T.tensor([state],dtype=T.float).to(self.actor.device)


.....saving models......
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
episode  0 score:  -31.59924170761155 avg score:  -31.59924170761155
[-0.1631567  -0.06659688]
episode  1 score:  -31.195416076320107 avg score:  -31.397328891965827
[0.5691051  0.02813734]
episode  2 score:  -29.91701443905644 avg score:  -30.903890740996033
[-0.64097506 -0.00368461]
episode  3 score:  -30.189457649004552 avg score:  -30.725282467998163
[0.4000836  0.00470653]
episode  4 score:  -31.143666879812276 avg score:  -30.808959350360986
[0.24884525 0.02019336]
episode  5 score:  -30.805424544677997 avg score:  -30.80837021608049
[ 0.25699818 -0.06280746]
episode  6 score:  -29.58099386017296 avg score:  -30.633030736665127
[-1.0846096   0.01441574]
episode  7 score:  -30.032696596702547 avg score:  -30.557988969169806
[-1.0330003  -0.00269347]
episode  8 score:  -30.03557415598263 avg score:  -30.499942878815673


episode  72 score:  -30.80999509696169 avg score:  -29.052605127087862
[-0.24416432 -0.01805981]
episode  73 score:  -31.43365130448283 avg score:  -29.08478142678239
[-0.604864   -0.01919538]
episode  74 score:  -30.235691920931608 avg score:  -29.100126900037708
[-0.57659775  0.03789165]
episode  75 score:  -33.16734952250613 avg score:  -29.15364298717545
[0.49182048 0.05012124]
episode  76 score:  -31.245396721443328 avg score:  -29.18080862008802
[-0.44981351 -0.0556794 ]
episode  77 score:  -30.34612277365309 avg score:  -29.19574854513372
[0.00126359 0.03032239]
episode  78 score:  -31.692362806804777 avg score:  -29.227351257306776
[-0.672322   -0.04357587]
episode  79 score:  -29.305703324795882 avg score:  -29.2283306581504
[-0.95067036 -0.06863715]
.....saving models......
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
episode  80 score:  -30.817069618995856 avg score:  -29.247944719

episode  145 score:  -30.776845967845816 avg score:  -30.495205009924245
[ 0.5082654  -0.03553743]
episode  146 score:  -30.080496695309503 avg score:  -30.505401362488918
[-0.9158256  -0.04980589]
episode  147 score:  -30.107915458327998 avg score:  -30.503652950254413
[-0.9649609  -0.05395727]
episode  148 score:  -32.63695823098884 avg score:  -30.518867960053303
[-0.9692432 -0.0640692]
episode  149 score:  -30.046505478507207 avg score:  -30.535208535658832
[ 0.5413571 -0.0059736]
.....saving models......
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
episode  150 score:  -30.77089523416766 avg score:  -30.551639487531073
[-0.9149493  0.0579451]
episode  151 score:  -30.267631088146945 avg score:  -30.5366326253004
[0.5183623  0.06360836]
episode  152 score:  -29.960854576719488 avg score:  -30.529148740442356
[0.0758213  0.04626741]
episode  153 score:  -29.86707482164044 avg score:  -30.5

...

episode  5401 score:  -29.61265874134045 avg score:  -30.533143062967216
[0.20302352 0.03420443]
episode  5402 score:  -30.45192482382326 avg score:  -30.518303366322307
[0.17429332 0.04908588]
episode  5403 score:  -32.055952441718034 avg score:  -30.54179795926744
[-0.17965092 -0.05191534]
episode  5404 score:  -31.012635707792914 avg score:  -30.553148940831292
[-1.0097563   0.05377813]
episode  5405 score:  -30.77930857424273 avg score:  -30.555510459490552
[-0.00356655 -0.03099495]
episode  5406 score:  -31.17410817268676 avg score:  -30.55662810279569
[ 0.22998944 -0.06803648]
episode  5407 score:  -31.359140314965853 avg score:  -30.56498694403832
[-1.1943485e+00 -6.1667076e-04]
episode  5408 score:  -30.564607608807606 avg score:  -30.555430231829487
[-0.5515579  -0.06474197]
episode  5409 score:  -29.861100008520708 avg score:  -30.554851058324648
[-0.21520019 -0.04026135]
.....saving models......
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint..

episode  5473 score:  -30.233543558316395 avg score:  -30.526572281729955
[-0.95429003 -0.03172611]
episode  5474 score:  -30.542541928992023 avg score:  -30.528452792533017
[-1.1693857   0.01017264]
episode  5475 score:  -31.222651768187642 avg score:  -30.53760972300768
[-0.8806102   0.00964356]
episode  5476 score:  -29.65559144247039 avg score:  -30.530544301025333
[-0.49470088 -0.04279659]
episode  5477 score:  -30.846851313549244 avg score:  -30.52525437375324
[-0.58581674 -0.03527419]
episode  5478 score:  -30.77247416619165 avg score:  -30.521320599209975
[0.40127128 0.0307602 ]
episode  5479 score:  -29.299708531848484 avg score:  -30.508940379478034
[ 0.5846478  -0.02939407]
.....saving models......
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
episode  5480 score:  -31.06431824957368 avg score:  -30.528448140402283
[0.5688662  0.05962583]
episode  5481 score:  -30.79783102294055 avg

episode  5545 score:  -29.934956248338924 avg score:  -29.531536878993002
[-0.19389097 -0.01341143]
episode  5546 score:  -28.87214878993269 avg score:  -29.50494757305767
[-0.1198414  -0.03328613]
episode  5547 score:  -29.78705743100566 avg score:  -29.49568490256263
[-0.8389715   0.03740258]
episode  5548 score:  -29.31106378018317 avg score:  -29.481583470876025
[-0.71682715  0.00476215]
episode  5549 score:  -29.98841305604635 avg score:  -29.474497883433
[ 0.31276157 -0.01498431]
.....saving models......
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
episode  5550 score:  -32.49445302699131 avg score:  -29.500356223358402
[-1.087479   -0.04915849]
episode  5551 score:  -31.375912443278843 avg score:  -29.517218531362325
[-0.26657927 -0.04235926]
episode  5552 score:  -30.94051998335554 avg score:  -29.531401946684422
[-0.7623183  -0.02237082]
episode  5553 score:  -30.903235745291763 avg 

episode  5617 score:  -29.70192362224595 avg score:  -29.53893069881872
[ 0.36785775 -0.01599468]
episode  5618 score:  -29.31400783820453 avg score:  -29.535981026077305
[-0.48470443 -0.00104783]
episode  5619 score:  -30.198009402767642 avg score:  -29.534227989316154
[-0.14111704  0.04103125]
.....saving models......
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
.....saving checkpoint.....
episode  5620 score:  -30.846464192287183 avg score:  -29.523357965133524
[0.268762   0.06006227]
episode  5621 score:  -30.76071673042888 avg score:  -29.522147855233655
[-0.41302517  0.04397354]
episode  5622 score:  -30.39100602696075 avg score:  -29.509312418057387
[-0.38801685  0.02729886]
episode  5623 score:  -30.741180471174864 avg score:  -29.519955292893847
[-0.36297935  0.02655087]
episode  5624 score:  -30.57599753177692 avg score:  -29.514920391724573
[0.49276397 0.01197945]
episode  5625 score:  -30.225808437258337 avg

KeyboardInterrupt: 

In [ ]:
plt.plot(score_history)
plt.figure()
plt.plot(avg_scores)

In [ ]:
import pybullet as p
import gym

#env = InvertedPendulum()
env = gym.make('MountainCarContinuous-v0')
#env = gym.make("BipedalWalker-v3")
#env = gym.make('InvertedDoublePendulumBulletEnv-v0')


load_checkpoint = True
agent = SACAgent(input_dims = env.observation_space.shape, env =env,
                n_actions = env.action_space.shape[0])
if load_checkpoint:
    agent.load_models()
p.connect(p.DIRECT)
for i in range(1):
    obs = env.reset()
    score = 0
    reward =0
    done = False
    
    while not done:
        act = agent.choose_action(obs)
        new_state, reward, done, info = env.step(act)        
        #env.render()
        env.render()
        obs = new_state
        score += reward
    print(score)
if env.viewer is not None:
    glfw.destroy_window(env.viewer.window)
    env.viewer = None
env.close()

In [ ]:
!pip install glfw